In [2]:
import pandas as pd
import csv
import sys
import requests
from lxml import html as htmlRenderer
csv.field_size_limit(sys.maxsize)

131072

In [3]:
csv_file_in = "/home/cflores/cflores_workspace/comments-retriever/results/elpais/elpais-01012019-15092019-contents.csv"
csv_file_out = "/home/cflores/cflores_workspace/comments-retriever/results/elpais/elpais-01012019-15092019-contents-tmp.csv"

In [4]:
def is_valid_csv(csv_file):
    invalid_rows = []
    no_content = []
    no_title = []
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        counter = 0
        for r in reader:
            if not r[0]:
                print(" row -{}- row has not url.".format(counter))
                invalid_rows.append(counter)
            if not r[1]: 
                print(" row -{}- row has not content.".format(counter))
                invalid_rows.append(counter)
                no_content.append(r[0])
            if not r[2] or r[2] == r[0]:
                print(" row -{}- row has not title.".format(counter))
                invalid_rows.append(counter)
                no_title.append(r[0])
            counter = counter + 1
        
    return len(list(dict.fromkeys(invalid_rows))) == 0, invalid_rows, no_content, no_title

In [5]:
valid, invalid_rows, no_content, no_title = is_valid_csv(csv_file_in)

 row -29- row has not content.
 row -40- row has not content.
 row -72- row has not content.
 row -97- row has not content.
 row -166- row has not content.
 row -185- row has not content.
 row -207- row has not content.
 row -221- row has not content.
 row -243- row has not content.
 row -294- row has not content.
 row -328- row has not content.
 row -364- row has not content.
 row -394- row has not content.
 row -430- row has not content.
 row -473- row has not content.
 row -512- row has not content.
 row -564- row has not title.
 row -570- row has not content.
 row -630- row has not content.
 row -631- row has not content.
 row -647- row has not content.
 row -667- row has not content.
 row -698- row has not title.
 row -701- row has not content.
 row -710- row has not content.
 row -722- row has not content.
 row -725- row has not content.
 row -767- row has not content.
 row -774- row has not content.
 row -839- row has not content.
 row -870- row has not content.
 row -879- row h

In [6]:
len(no_content)

472

In [12]:
len(no_title)

8

In [8]:
no_title

['https://brasil.elpais.com/brasil/2019/01/13/politica/1547367596_725724.html',
 'https://brasil.elpais.com/brasil/2019/01/15/ciencia/1547542171_994570.html',
 'https://brasil.elpais.com/brasil/2019/02/11/politica/1549900545_825737.html',
 'https://brasil.elpais.com/brasil/2019/07/07/politica/1562502904_916776.html',
 'https://brasil.elpais.com/brasil/2019/08/07/politica/1565182720_327306.html']

In [7]:
no_content

['https://elpais.com/elpais/2019/01/01/album/1546355344_412589.html',
 'https://elpais.com/elpais/2019/01/02/album/1546428458_457534.html',
 'https://elpais.com/elpais/2019/01/03/album/1546498809_839576.html',
 'https://elpais.com/elpais/2019/01/03/album/1546514563_874937.html',
 'https://elpais.com/elpais/2019/01/04/album/1546608102_806879.html',
 'https://elpais.com/elpais/2019/01/05/album/1546703847_118703.html',
 'https://elpais.com/elpais/2019/01/06/album/1546778065_952915.html',
 'https://elpais.com/elpais/2019/01/07/la_voz_de_inaki/1546847697_268615.html',
 'https://elpais.com/elpais/2019/01/07/album/1546847484_699881.html',
 'https://elpais.com/elpais/2019/01/08/album/1546943119_938295.html',
 'https://elpais.com/elpais/2019/01/09/la_voz_de_inaki/1547020472_269335.html',
 'https://elpais.com/elpais/2019/01/09/media/1547062134_734790.html',
 'https://elpais.com/elpais/2019/01/10/la_voz_de_inaki/1547106772_946877.html',
 'https://elpais.com/elpais/2019/01/10/album/1547123881_0832

In [17]:
def extract_content(url=None):
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    queries_xpath = ["//span[@class='cuerpo-texto ']//p",
                     "//span[@class='cuerpo-texto  unreg']/p",
                     "//span[@class='cuerpo-texto']/p"]
    contentStr = ""
    for q in queries_xpath:
        commentsElList = renderedPage.xpath(q)
        if len(commentsElList) > 0:
            contentArr = []
            for p in commentsElList:
                contentArr.append(p.text_content())
            contentStr = "".join([parrafo for parrafo in contentArr])
            break
    if not contentStr:
        print("\t -> url has not content found {}".format(url))
    return contentStr

In [16]:
def get_title(url):
    queryXpath = "//article//h1"
    queries_xpath = ["//span[@class='titular']",
                     "//h1[@class='titular']", 
                     "//h1[@class='titular principal']"]
    response = requests.get(url)
    renderedPage = htmlRenderer.fromstring(response.text)
    el = renderedPage.xpath(queryXpath)
    title = url
    for q in queries_xpath:
        el = renderedPage.xpath(q)
        if len(el) > 0:
            title = el[0].text if el[0].text != None else el[0].text_content()
            break
    if title == url:
        print (" \t -> title not found for url {}".format(url))
    return title

    

In [ ]:
keys = []
no_content = []
no_title = []
with open(csv_file_in, 'r') as f:
    with open(csv_file_out, 'w') as fout:
        writer = csv.writer(fout)
        reader = csv.reader(f)
        for r in reader:
            if not r[1]:
                # no content
                print(" => row with url {} has any content".format(r[0]))
                content = extract_content(r[0])
                if content:
                    print(" \t ===>>> content found ")
                r[1] = content
            else:
                r[1] = r[1].replace("\t", "").replace("\n", "").lstrip().rstrip()
            if not r[2] or r[2] == r[0]:
                # no title
                print(" => row with url {} has any title".format(r[0]))
                r[2] = get_title(r[0])
            
            writer.writerow(r)

 => row with url https://www.abc.es/espana/abci-sera-noticia-2019-201901010445_noticia.html has any content
	 -> url has not content found https://www.abc.es/espana/abci-sera-noticia-2019-201901010445_noticia.html
 => row with url https://www.abc.es/play/television/noticias/abci-vestido-cristina-pedroche-cristina-pedroche-o-triunfo-audiencia-linchamiento-mediatico-201901010044_noticia.html has any content
	 -> url has not content found https://www.abc.es/play/television/noticias/abci-vestido-cristina-pedroche-cristina-pedroche-o-triunfo-audiencia-linchamiento-mediatico-201901010044_noticia.html
 => row with url https://www.abc.es/play/series/noticias/abci-series-podras-este-enero-201901010251_noticia.html has any content
	 -> url has not content found https://www.abc.es/play/series/noticias/abci-series-podras-este-enero-201901010251_noticia.html
 => row with url https://www.abc.es/opinion/abci-fuerzas-armadas-y-pueblo-espanol-201901010213_noticia.html has any content
 	 ===>>> content 

In [ ]:
valid, invalid_rows, no_content, no_title = is_valid_csv(csv_file_out)